---
layout: post  
---

While working on [a previous post about kmer sizes, error rates, and coverage]({{ site.baseurl }}{% post_url 2020-12-19-observed-kmers-vs-error-rate %}), I found a type-stability issue in a function that I was writing.

The conceptual issue is described [here](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-value-type) in the Julia language manual.

The core of the problem is that Julia's compilation is centered around the _types_ of the inputs, not their values

Let's demonstrate this through a few examples

In [1]:
import Pkg
pkgs = [
    "BioSequences",
    "Random",
    "BenchmarkTools",
    "Primes"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
# This macro intercepts the output that @code_warntype would write to the screen
# Because this file is originally generated in a Jupyter notebook,
# we have a rich multimedia display with text coloration options
# @code_warntype uses color to emphasize areas of concern in the functions it analyzes
# that color emphasis renders in the Jupyter notebooks,
# but not the jekyll html pages generated by the notebooks.
# By capturing the output, we also strip the output of its color,
# allowing it to render more cleanly on the jekyll pages
macro capture_code_warntype(x)
    quote
        eval(:(sprint($((@macroexpand $x).args...))))
    end
end

@capture_code_warntype (macro with 1 method)

Let's say we would like a kmer of length `3`

In [3]:
kmer = BioSequences.DNAMer("ACG")

DNA 3-mer:
ACG

While this kmer has a length of `3`, the actual data type of the kmer is something entirely different

In [4]:
typeof(kmer)

BioSequences.Mer{BioSequences.DNAAlphabet{2},3}

As shown above, the type of this length `3` kmer is a composite type consisting of the alphabet `BioSequences.DNAAlphabet{2}` and the length `3`

If we want a function that will return every `3`-mer in a sequence, passing an argument `kmer_size` of `3` is insufficient for the Julia language to unambiguously know the return type ahead of time, which would be `BioSequences.Mer{BioSequences.DNAAlphabet{2},3}`

The reason for this is that if we pass a _value_ of `k=3`, the datatype of `3` is an Integer

In [5]:
typeof(3)

Int64

An integer can hold any value between the minimum and maximum values that can be stored the datatype

In [6]:
typemin(Int):typemax(Int)

-9223372036854775808:9223372036854775807

While changing the value of an integer does not change the datatype, changing the value of k _DOES_ change the datatype of a kmer (as it is currently designed and implemented in the BioSequences codebase in Julia)

In [7]:
println("Integers:")
@show type_1 = typeof(3);
@show type_2 = typeof(4);
@show type_1 == type_2;

println("")
println("Kmers:")
@show type_1 = typeof(BioSequences.DNAMer("ACG"));
@show type_2 = typeof(BioSequences.DNAMer("ACGT"));
@show type_1 != type_2;

Integers:
type_1 = typeof(3) = Int64
type_2 = typeof(4) = Int64
type_1 == type_2 = true

Kmers:
type_1 = typeof(BioSequences.DNAMer("ACG")) = BioSequences.Mer{BioSequences.DNAAlphabet{2},3}
type_2 = typeof(BioSequences.DNAMer("ACGT")) = BioSequences.Mer{BioSequences.DNAAlphabet{2},4}
type_1 != type_2 = true


Let's demonstrate how this plays out in terms of Julia type-inference and runtime

First, we will set our k parameter and generate a random sequence

In [8]:
k = 3
sequence = BioSequences.randdnaseq(Random.seed!(1), 10)

10nt DNA Sequence:
TCGTCCCAGG

Next we will define a function that takes the k-size and sequence as inputs, and returns the set of all kmers observed as the output

In [9]:
function assess_kmers_1(k, sequence)
    KMER_TYPE = BioSequences.DNAMer{k}
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_1 (generic function with 1 method)

In [10]:
assess_kmers_1(k, sequence)

Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}} with 8 elements:
  ACG
  GAC
  CCC
  GGA
  CGA
  AGG
  CAG
  CCA

We can see that the function executed without error and returned our desired result, so there is no issue with correctness of this function

However, if we use Julia's built in `@code_warntype` to assess the type-inferring of the function, we can see the issue

In [23]:
# @code_warntype assess_kmers_1(k, sequence)
println(@capture_code_warntype @code_warntype assess_kmers_1(k, sequence))

Variables
  #self#::Core.Compiler.Const(assess_kmers_1, false)
  k::Int64
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  KMER_TYPE::TYPE{BIOSEQUENCES.MER{BIOSEQUENCES.DNAALPHABET{2},_A}} WHERE _A
  kmers::SET{_A} WHERE _A
  @_6::UNION{NOTHING, TUPLE{BIOSEQUENCES.MERITERRESULT,TUPLE{INT64,ANY,ANY,ANY}}}
  kmer_set::BIOSEQUENCES.MERITERRESULT

Body::SET{_A} WHERE _A
1 ─ %1  = BioSequences.DNAMer::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},K} where K, false)
│         (KMER_TYPE = Core.apply_type(%1, k))
│   %3  = Core.apply_type(Main.Set, KMER_TYPE)::TYPE{SET{_A}} WHERE _A
│         (kmers = (%3)())
│   %5  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %6  = KMER_TYPE::TYPE{BIOSEQUENCES.MER{BIOSEQUENCES.DNAALPHABET{2},_A}} WHERE _A
│   %7  = (%5)(%6, sequence)::BIOSEQUENCES.EVERYMERITERATOR{_A,BIOSEQUENCES.LONGSEQUENCE{BIOSEQUENCES.DNAALPHABET{4}}} WHERE _A<:BIOSEQUENCES.ABSTRACTMER
│         (@_6 = Base.iterate(%7))
│ 

We can see that the Body of the function has the return type `Body::SET{_A} WHERE _A`

The function knows it will return a Set, but it isn't sure what datatype(s) will be stored inside

As a result, the function has given this unknown datatype a variable, `_A`

Let's review the memory utilization and runtime of this function as a benchmark to see if we can improve upon it

In [12]:
BenchmarkTools.@benchmark assess_kmers_1(k, sequence)

BenchmarkTools.Trial: 
  memory estimate:  2.02 KiB
  allocs estimate:  54
  --------------
  minimum time:     5.337 μs (0.00% GC)
  median time:      5.624 μs (0.00% GC)
  mean time:        6.504 μs (2.30% GC)
  maximum time:     849.336 μs (98.66% GC)
  --------------
  samples:          10000
  evals/sample:     6

Here we can see that the function requires 54 individual allocations, requires 2 KiB of memory, and has a mean runtime of approximately 6 μs

To circumvent this issue, Julia has a built-in `Val` type that can be used to wrap integers to provide type-stability in functions. Let's try using it and seeing what happens

In [13]:
function assess_kmers_2(K::Val{k}, sequence) where k
    KMER_TYPE = BioSequences.DNAMer{k}
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_2 (generic function with 1 method)

In [14]:
assess_kmers_2(Val(k), sequence)

Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}} with 8 elements:
  ACG
  GAC
  CCC
  GGA
  CGA
  AGG
  CAG
  CCA

As shown above, this function generates the same output as the first function we wrote.

We can confirm this by running both functions again and directly comparing their outputs.

In [15]:
assess_kmers_1(k, sequence) == assess_kmers_2(Val(k), sequence)

true

Let's look at whether the type inference has improved

In [24]:
# @code_warntype assess_kmers_2(Val(k), sequence)
println(@capture_code_warntype @code_warntype assess_kmers_2(Val(k), sequence))

Variables
  #self#::Core.Compiler.Const(assess_kmers_2, false)
  K::Core.Compiler.Const(Val{3}(), false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  KMER_TYPE::Type{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_6::UNION{NOTHING, TUPLE{BIOSEQUENCES.MERITERRESULT{BIOSEQUENCES.MER{BIOSEQUENCES.DNAALPHABET{2},3}},TUPLE{INT64,INT64,UINT64,UINT64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = BioSequences.DNAMer::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},K} where K, false)
│         (KMER_TYPE = Core.apply_type(%1, $(Expr(:static_parameter, 1))))
│   %3  = Core.apply_type(Main.Set, KMER_TYPE::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false))::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers

Here we can see that the function body is able to properly infer the return type from the inputs, and knows that it will return a type `Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}` given the inputs

In [17]:
BenchmarkTools.@benchmark assess_kmers_2(Val(k), sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     2.530 μs (0.00% GC)
  median time:      2.593 μs (0.00% GC)
  mean time:        2.713 μs (1.32% GC)
  maximum time:     364.372 μs (98.25% GC)
  --------------
  samples:          10000
  evals/sample:     9

The benchmarking results have also improved. Memory allocation is ~4x lower, the number of allocations has decreased by >10x, and the mean runtime has decreased by >2x

While this improvement is nice, I'd like to avoid using the `Val` construct. The mixed usage of k as an integer in some areas of the code and as a `Val` type elsewhere would not be intuitive to those unfamiliar with this type-stability concern.

As an alternative, let's try creating a function that takes the desired kmer type directly

In [18]:
function assess_kmers_3(::Type{KMER_TYPE}, sequence) where KMER_TYPE
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_3 (generic function with 1 method)

In [19]:
assess_kmers_3(BioSequences.DNAMer{k}, sequence)

Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}} with 8 elements:
  ACG
  GAC
  CCC
  GGA
  CGA
  AGG
  CAG
  CCA

We can see that this function also generates the same output as the others

In [20]:
assess_kmers_1(k, sequence) == assess_kmers_2(Val(k), sequence) == assess_kmers_3(BioSequences.DNAMer{k}, sequence)

true

Like `assess_kmers_2`, this function is able to correctly infer the return type based on the types of the inputs

In [25]:
# @code_warntype assess_kmers_3(BioSequences.DNAMer{k}, sequence)
println(@capture_code_warntype @code_warntype assess_kmers_3(BioSequences.DNAMer{k}, sequence))

Variables
  #self#::Core.Compiler.Const(assess_kmers_3, false)
  #unused#::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_5::UNION{NOTHING, TUPLE{BIOSEQUENCES.MERITERRESULT{BIOSEQUENCES.MER{BIOSEQUENCES.DNAALPHABET{2},3}},TUPLE{INT64,INT64,UINT64,UINT64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = Core.apply_type(Main.Set, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers = (%1)())
│   %3  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %4  = $(Expr(:static_parameter, 1))::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
│   %5  = (%3)(%4, sequence)::BioSequences.EveryMerIterator{BioSe

For reasons I don't fully understand, this function variant is also much faster

In [22]:
BenchmarkTools.@benchmark assess_kmers_3(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     427.779 ns (0.00% GC)
  median time:      442.156 ns (0.00% GC)
  mean time:        512.207 ns (7.19% GC)
  maximum time:     28.346 μs (98.16% GC)
  --------------
  samples:          10000
  evals/sample:     199

Here we can see that the number of allocations has stayed the same as our previous method, but something about directly providing the desired return type has enabled us to further reduce our runtime by ~5x over the `assess_kmers_2` method, and by over 10x relative to our original `assess_kmers_1` method

While the original goal of `assess_kmers_3` relative to `assess_kmers_2` was clarity, rather than efficiency, we seem to have also made the function more efficient as a welcome side effect